# 대한민국 출생아수 및 합계출산율 데이터 분석

이 노트북은 대한민국의 출생아수와 합계출산율 데이터를 클렌징하고 분석하는 과정을 담고 있습니다.

## 목차
1. 라이브러리 import 및 데이터 로드
2. 데이터 구조 및 기본 정보 확인
3. 결측값 처리
4. 데이터 타입 변환 및 정리
5. 이상값 탐지 및 처리
6. 시계열 데이터 분석
7. 출생아수 트렌드 시각화
8. 합계출산율 변화 분석
9. 상관관계 분석
10. 통계적 요약 및 인사이트 도출

## 1. 라이브러리 import 및 데이터 로드

필요한 라이브러리들을 import하고 Excel 데이터 파일을 로드합니다.

In [ ]:
# 필요한 라이브러리 import
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime

# 한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 경고 메시지 숨기기
warnings.filterwarnings('ignore')

# 그래프 스타일 설정
sns.set_style("whitegrid")
plt.style.use('default')

print("라이브러리 import 완료!")

In [ ]:
# Excel 파일 로드
file_path = r'c:\work\출생아수__합계출산율__자연증가_등_20250808152552.xlsx'

try:
    # Excel 파일의 모든 시트 이름 확인
    excel_file = pd.ExcelFile(file_path)
    print("시트 이름들:", excel_file.sheet_names)
    
    # 첫 번째 시트 로드 (또는 적절한 시트 선택)
    df = pd.read_excel(file_path, sheet_name=0)
    print(f"\n데이터 로드 완료! 형태: {df.shape}")
    print("첫 5행 데이터:")
    print(df.head())
    
except Exception as e:
    print(f"파일 로드 중 오류 발생: {e}")
    # 만약 파일이 없다면 샘플 데이터 생성
    print("샘플 데이터를 생성합니다...")
    years = list(range(2000, 2025))
    birth_count = np.random.randint(400000, 600000, len(years))
    fertility_rate = np.random.uniform(0.8, 1.5, len(years))
    
    df = pd.DataFrame({
        '연도': years,
        '출생아수': birth_count,
        '합계출산율': fertility_rate
    })
    print("샘플 데이터 생성 완료!")

## 2. 데이터 구조 및 기본 정보 확인

데이터의 기본 구조와 정보를 파악합니다.

In [ ]:
# 데이터 기본 정보
print("=== 데이터 기본 정보 ===")
print(f"데이터 크기: {df.shape}")
print(f"컬럼 개수: {len(df.columns)}")
print(f"행 개수: {len(df)}")

print("\n=== 컬럼 이름 ===")
for i, col in enumerate(df.columns):
    print(f"{i+1}. {col}")

print("\n=== 데이터 타입 ===")
print(df.dtypes)

print("\n=== 기본 통계 정보 ===")
print(df.describe())

In [ ]:
# 상세 데이터 정보
print("=== 데이터 상세 정보 ===")
print(df.info())

print("\n=== 데이터 미리보기 (처음 10행) ===")
display(df.head(10))

print("\n=== 데이터 미리보기 (마지막 5행) ===")
display(df.tail())

## 3. 결측값 처리

데이터의 결측값을 확인하고 적절한 방법으로 처리합니다.

In [ ]:
# 결측값 확인
print("=== 결측값 확인 ===")
missing_values = df.isnull().sum()
print("각 컬럼별 결측값 개수:")
print(missing_values)

print(f"\n전체 결측값 개수: {df.isnull().sum().sum()}")
print(f"전체 데이터 대비 결측값 비율: {(df.isnull().sum().sum() / (len(df) * len(df.columns))) * 100:.2f}%")

# 결측값이 있는 행 확인
if df.isnull().any().any():
    print("\n=== 결측값이 있는 행들 ===")
    missing_rows = df[df.isnull().any(axis=1)]
    display(missing_rows)
    
    # 결측값 시각화
    plt.figure(figsize=(10, 6))
    sns.heatmap(df.isnull(), cbar=True, yticklabels=False, cmap='viridis')
    plt.title('결측값 히트맵 (노란색: 결측값)')
    plt.show()
else:
    print("\n결측값이 없습니다!")

In [ ]:
# 결측값 처리
print("=== 결측값 처리 ===")

# 데이터 복사본 생성
df_cleaned = df.copy()

# 결측값이 있는 경우 처리 방법 선택
if df.isnull().any().any():
    # 1. 수치형 데이터는 평균값으로 대체
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    for col in numeric_columns:
        if df[col].isnull().any():
            mean_value = df[col].mean()
            df_cleaned[col].fillna(mean_value, inplace=True)
            print(f"{col} 컬럼의 결측값을 평균값 {mean_value:.2f}로 대체했습니다.")
    
    # 2. 문자형 데이터는 최빈값으로 대체
    categorical_columns = df.select_dtypes(include=['object']).columns
    for col in categorical_columns:
        if df[col].isnull().any():
            mode_value = df[col].mode().iloc[0] if not df[col].mode().empty else "Unknown"
            df_cleaned[col].fillna(mode_value, inplace=True)
            print(f"{col} 컬럼의 결측값을 최빈값 '{mode_value}'로 대체했습니다.")
    
    print(f"\n처리 후 결측값 개수: {df_cleaned.isnull().sum().sum()}")
else:
    print("결측값이 없어 별도 처리가 필요하지 않습니다.")

# 처리 결과 확인
print("\n=== 결측값 처리 완료 ===")
print("처리 후 데이터 형태:", df_cleaned.shape)

## 4. 데이터 타입 변환 및 정리

컬럼명을 정리하고 데이터 타입을 적절하게 변환합니다.

In [ ]:
# 컬럼명 정리
print("=== 컬럼명 정리 ===")
print("기존 컬럼명:")
for i, col in enumerate(df_cleaned.columns):
    print(f"{i+1}. '{col}'")

# 컬럼명에서 공백 제거 및 표준화
df_cleaned.columns = df_cleaned.columns.str.strip()

# 주요 컬럼 식별 및 표준화
column_mapping = {}
for col in df_cleaned.columns:
    col_lower = col.lower()
    if '연도' in col or 'year' in col_lower:
        column_mapping[col] = '연도'
    elif '출생' in col and '수' in col:
        column_mapping[col] = '출생아수'
    elif '합계출산율' in col or '출산율' in col:
        column_mapping[col] = '합계출산율'
    elif '자연증가' in col:
        column_mapping[col] = '자연증가율'

# 컬럼명 변경
if column_mapping:
    df_cleaned.rename(columns=column_mapping, inplace=True)
    print(f"\n컬럼명이 변경되었습니다: {column_mapping}")

print("\n변경된 컬럼명:")
for i, col in enumerate(df_cleaned.columns):
    print(f"{i+1}. '{col}'")

In [ ]:
# 데이터 타입 변환
print("=== 데이터 타입 변환 ===")
print("변환 전 데이터 타입:")
print(df_cleaned.dtypes)

# 연도 컬럼이 있다면 정수형으로 변환
if '연도' in df_cleaned.columns:
    df_cleaned['연도'] = pd.to_numeric(df_cleaned['연도'], errors='coerce').astype('Int64')
    print("연도 컬럼을 정수형으로 변환했습니다.")

# 출생아수 컬럼이 있다면 정수형으로 변환
if '출생아수' in df_cleaned.columns:
    # 쉼표나 기타 문자 제거 후 숫자로 변환
    if df_cleaned['출생아수'].dtype == 'object':
        df_cleaned['출생아수'] = df_cleaned['출생아수'].astype(str).str.replace(',', '').str.replace(' ', '')
    df_cleaned['출생아수'] = pd.to_numeric(df_cleaned['출생아수'], errors='coerce').astype('Int64')
    print("출생아수 컬럼을 정수형으로 변환했습니다.")

# 합계출산율 컬럼이 있다면 실수형으로 변환
if '합계출산율' in df_cleaned.columns:
    df_cleaned['합계출산율'] = pd.to_numeric(df_cleaned['합계출산율'], errors='coerce').astype('float64')
    print("합계출산율 컬럼을 실수형으로 변환했습니다.")

print("\n변환 후 데이터 타입:")
print(df_cleaned.dtypes)

# 변환 결과 확인
print("\n=== 변환 결과 미리보기 ===")
display(df_cleaned.head())

## 5. 이상값 탐지 및 처리

박스플롯과 IQR 방법을 사용하여 이상값을 탐지하고 처리합니다.

In [ ]:
# 이상값 탐지 함수 정의
def detect_outliers_iqr(data, column):
    """IQR 방법으로 이상값 탐지"""
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    outliers = data[(data[column] < lower_bound) | (data[column] > upper_bound)]
    return outliers, lower_bound, upper_bound

# 수치형 컬럼에 대해 이상값 탐지
numeric_cols = df_cleaned.select_dtypes(include=[np.number]).columns

print("=== 이상값 탐지 ===")
for col in numeric_cols:
    if col != '연도':  # 연도는 이상값 검사에서 제외
        outliers, lower, upper = detect_outliers_iqr(df_cleaned, col)
        print(f"\n{col} 컬럼:")
        print(f"  정상 범위: {lower:.2f} ~ {upper:.2f}")
        print(f"  이상값 개수: {len(outliers)}")
        
        if len(outliers) > 0:
            print(f"  이상값:")
            for idx, row in outliers.iterrows():
                print(f"    {row['연도'] if '연도' in df_cleaned.columns else idx}: {row[col]}")

# 박스플롯으로 이상값 시각화
fig, axes = plt.subplots(1, len(numeric_cols)-1 if '연도' in numeric_cols else len(numeric_cols), 
                        figsize=(15, 5))
if len(numeric_cols) == 1:
    axes = [axes]

plot_idx = 0
for col in numeric_cols:
    if col != '연도':
        if len(numeric_cols) > 2:
            ax = axes[plot_idx]
        else:
            ax = axes if len(numeric_cols) == 2 else axes[0]
        
        ax.boxplot(df_cleaned[col].dropna())
        ax.set_title(f'{col} 박스플롯')
        ax.set_ylabel(col)
        plot_idx += 1

plt.tight_layout()
plt.show()

## 6. 시계열 데이터 분석

연도별 데이터의 시계열 특성을 분석합니다.

In [ ]:
# 시계열 데이터 정렬
if '연도' in df_cleaned.columns:
    df_sorted = df_cleaned.sort_values('연도').reset_index(drop=True)
    
    print("=== 시계열 데이터 기본 정보 ===")
    print(f"데이터 기간: {df_sorted['연도'].min()} ~ {df_sorted['연도'].max()}")
    print(f"총 기간: {df_sorted['연도'].max() - df_sorted['연도'].min() + 1}년")
    print(f"데이터 포인트: {len(df_sorted)}개")
    
    # 연도별 데이터 연속성 확인
    years = df_sorted['연도'].values
    missing_years = []
    for i in range(int(years.min()), int(years.max()) + 1):
        if i not in years:
            missing_years.append(i)
    
    if missing_years:
        print(f"누락된 연도: {missing_years}")
    else:
        print("연도 데이터가 연속적입니다.")
    
    # 시계열 변화율 계산 (연도별)
    if '출생아수' in df_sorted.columns:
        df_sorted['출생아수_변화율'] = df_sorted['출생아수'].pct_change() * 100
        print(f"\n출생아수 평균 연간 변화율: {df_sorted['출생아수_변화율'].mean():.2f}%")
    
    if '합계출산율' in df_sorted.columns:
        df_sorted['합계출산율_변화율'] = df_sorted['합계출산율'].pct_change() * 100
        print(f"합계출산율 평균 연간 변화율: {df_sorted['합계출산율_변화율'].mean():.2f}%")
    
    display(df_sorted.head(10))
else:
    print("연도 컬럼이 없어 시계열 분석을 수행할 수 없습니다.")
    df_sorted = df_cleaned

## 7. 출생아수 트렌드 시각화

연도별 출생아수 변화를 다양한 방법으로 시각화합니다.

In [ ]:
# 출생아수 트렌드 시각화
if '출생아수' in df_sorted.columns and '연도' in df_sorted.columns:
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. 기본 선 그래프
    axes[0, 0].plot(df_sorted['연도'], df_sorted['출생아수'], marker='o', linewidth=2, markersize=4)
    axes[0, 0].set_title('연도별 출생아수 변화', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('연도')
    axes[0, 0].set_ylabel('출생아수 (명)')
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 막대 그래프
    axes[0, 1].bar(df_sorted['연도'], df_sorted['출생아수'], alpha=0.7, color='skyblue')
    axes[0, 1].set_title('연도별 출생아수 (막대그래프)', fontsize=14, fontweight='bold')
    axes[0, 1].set_xlabel('연도')
    axes[0, 1].set_ylabel('출생아수 (명)')
    axes[0, 1].tick_params(axis='x', rotation=45)
    
    # 3. 변화율 그래프 (있는 경우)
    if '출생아수_변화율' in df_sorted.columns:
        axes[1, 0].plot(df_sorted['연도'][1:], df_sorted['출생아수_변화율'][1:], 
                       marker='s', color='red', linewidth=2)
        axes[1, 0].axhline(y=0, color='black', linestyle='--', alpha=0.5)
        axes[1, 0].set_title('연도별 출생아수 변화율', fontsize=14, fontweight='bold')
        axes[1, 0].set_xlabel('연도')
        axes[1, 0].set_ylabel('변화율 (%)')
        axes[1, 0].grid(True, alpha=0.3)
    
    # 4. 히스토그램
    axes[1, 1].hist(df_sorted['출생아수'], bins=15, alpha=0.7, color='green', edgecolor='black')
    axes[1, 1].set_title('출생아수 분포', fontsize=14, fontweight='bold')
    axes[1, 1].set_xlabel('출생아수 (명)')
    axes[1, 1].set_ylabel('빈도')
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # 주요 통계 정보
    print("=== 출생아수 주요 통계 ===")
    print(f"최대값: {df_sorted['출생아수'].max():,}명 ({df_sorted[df_sorted['출생아수'] == df_sorted['출생아수'].max()]['연도'].values[0]}년)")
    print(f"최소값: {df_sorted['출생아수'].min():,}명 ({df_sorted[df_sorted['출생아수'] == df_sorted['출생아수'].min()]['연도'].values[0]}년)")
    print(f"평균값: {df_sorted['출생아수'].mean():,.0f}명")
    print(f"표준편차: {df_sorted['출생아수'].std():,.0f}명")
else:
    print("출생아수 또는 연도 데이터가 없어 시각화할 수 없습니다.")

## 8. 합계출산율 변화 분석

합계출산율의 시간에 따른 변화를 분석하고 시각화합니다.

In [ ]:
# 합계출산율 변화 분석
if '합계출산율' in df_sorted.columns and '연도' in df_sorted.columns:
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # 1. 합계출산율 트렌드
    axes[0, 0].plot(df_sorted['연도'], df_sorted['합계출산율'], 
                   marker='o', linewidth=3, markersize=5, color='purple')
    axes[0, 0].axhline(y=2.1, color='red', linestyle='--', alpha=0.7, label='인구대체수준 (2.1)')
    axes[0, 0].axhline(y=1.3, color='orange', linestyle='--', alpha=0.7, label='초저출산 기준 (1.3)')
    axes[0, 0].set_title('연도별 합계출산율 변화', fontsize=14, fontweight='bold')
    axes[0, 0].set_xlabel('연도')
    axes[0, 0].set_ylabel('합계출산율')
    axes[0, 0].legend()
    axes[0, 0].grid(True, alpha=0.3)
    
    # 2. 출생아수와 합계출산율 듀얼 축 그래프
    if '출생아수' in df_sorted.columns:
        ax1 = axes[0, 1]
        ax2 = ax1.twinx()
        
        line1 = ax1.plot(df_sorted['연도'], df_sorted['출생아수'], 
                        color='blue', marker='o', label='출생아수')
        line2 = ax2.plot(df_sorted['연도'], df_sorted['합계출산율'], 
                        color='red', marker='s', label='합계출산율')
        
        ax1.set_xlabel('연도')
        ax1.set_ylabel('출생아수 (명)', color='blue')
        ax2.set_ylabel('합계출산율', color='red')
        ax1.set_title('출생아수와 합계출산율 비교', fontsize=14, fontweight='bold')
        
        # 범례 통합
        lines = line1 + line2
        labels = [l.get_label() for l in lines]
        ax1.legend(lines, labels, loc='upper right')
        ax1.grid(True, alpha=0.3)
    
    # 3. 합계출산율 변화율
    if '합계출산율_변화율' in df_sorted.columns:
        axes[1, 0].plot(df_sorted['연도'][1:], df_sorted['합계출산율_변화율'][1:], 
                       marker='d', color='green', linewidth=2)
        axes[1, 0].axhline(y=0, color='black', linestyle='--', alpha=0.5)
        axes[1, 0].set_title('연도별 합계출산율 변화율', fontsize=14, fontweight='bold')
        axes[1, 0].set_xlabel('연도')
        axes[1, 0].set_ylabel('변화율 (%)')
        axes[1, 0].grid(True, alpha=0.3)
    
    # 4. 합계출산율 구간별 분석
    fertility_ranges = pd.cut(df_sorted['합계출산율'], 
                             bins=[0, 1.0, 1.3, 1.8, 2.1, 3.0], 
                             labels=['극저출산(~1.0)', '초저출산(1.0~1.3)', 
                                   '저출산(1.3~1.8)', '적정(1.8~2.1)', '높음(2.1~)'])
    
    range_counts = fertility_ranges.value_counts().sort_index()
    axes[1, 1].pie(range_counts.values, labels=range_counts.index, autopct='%1.1f%%', 
                   startangle=90, colors=['darkred', 'red', 'orange', 'lightgreen', 'green'])
    axes[1, 1].set_title('합계출산율 구간별 분포', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # 주요 통계 정보
    print("=== 합계출산율 주요 통계 ===")
    print(f"최대값: {df_sorted['합계출산율'].max():.3f} ({df_sorted[df_sorted['합계출산율'] == df_sorted['합계출산율'].max()]['연도'].values[0]}년)")
    print(f"최소값: {df_sorted['합계출산율'].min():.3f} ({df_sorted[df_sorted['합계출산율'] == df_sorted['합계출산율'].min()]['연도'].values[0]}년)")
    print(f"평균값: {df_sorted['합계출산율'].mean():.3f}")
    print(f"표준편차: {df_sorted['합계출산율'].std():.3f}")
    
    # 출산율 수준별 기간 분석
    print("\n=== 출산율 수준별 기간 ===")
    for label, count in range_counts.items():
        print(f"{label}: {count}년")
        
else:
    print("합계출산율 또는 연도 데이터가 없어 시각화할 수 없습니다.")

## 9. 상관관계 분석

출생아수와 합계출산율 간의 상관관계를 분석합니다.

In [ ]:
# 상관관계 분석
# 수치형 컬럼만 선택
numeric_columns = df_sorted.select_dtypes(include=[np.number]).columns
correlation_data = df_sorted[numeric_columns]

print("=== 상관관계 분석 ===")
correlation_matrix = correlation_data.corr()
print("상관관계 매트릭스:")
print(correlation_matrix.round(3))

# 상관관계 히트맵
plt.figure(figsize=(12, 8))
mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
sns.heatmap(correlation_matrix, 
            mask=mask,
            annot=True, 
            cmap='RdBu_r', 
            center=0,
            square=True,
            linewidths=0.5,
            cbar_kws={"shrink": .8},
            fmt='.3f')
plt.title('변수 간 상관관계 히트맵', fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

# 출생아수와 합계출산율 간 상관관계 상세 분석
if '출생아수' in df_sorted.columns and '합계출산율' in df_sorted.columns:
    correlation_coef = df_sorted['출생아수'].corr(df_sorted['합계출산율'])
    print(f"\n출생아수와 합계출산율 간 상관계수: {correlation_coef:.4f}")
    
    # 상관관계 해석
    if abs(correlation_coef) >= 0.8:
        strength = "매우 강한"
    elif abs(correlation_coef) >= 0.6:
        strength = "강한"
    elif abs(correlation_coef) >= 0.4:
        strength = "중간"
    elif abs(correlation_coef) >= 0.2:
        strength = "약한"
    else:
        strength = "매우 약한"
    
    direction = "양의" if correlation_coef > 0 else "음의"
    print(f"해석: {strength} {direction} 상관관계")
    
    # 산점도
    plt.figure(figsize=(10, 6))
    plt.scatter(df_sorted['합계출산율'], df_sorted['출생아수'], alpha=0.7, s=60)
    
    # 추세선 추가
    z = np.polyfit(df_sorted['합계출산율'], df_sorted['출생아수'], 1)
    p = np.poly1d(z)
    plt.plot(df_sorted['합계출산율'], p(df_sorted['합계출산율']), "r--", alpha=0.8, linewidth=2)
    
    plt.xlabel('합계출산율')
    plt.ylabel('출생아수 (명)')
    plt.title(f'출생아수 vs 합계출산율 산점도 (상관계수: {correlation_coef:.4f})', 
              fontsize=14, fontweight='bold')
    plt.grid(True, alpha=0.3)
    
    # 연도별 색상 구분 (가능한 경우)
    if '연도' in df_sorted.columns:
        scatter = plt.scatter(df_sorted['합계출산율'], df_sorted['출생아수'], 
                            c=df_sorted['연도'], cmap='viridis', alpha=0.7, s=60)
        plt.colorbar(scatter, label='연도')
    
    plt.tight_layout()
    plt.show()

## 10. 통계적 요약 및 인사이트 도출

주요 통계 지표를 계산하고 데이터에서 발견한 인사이트를 정리합니다.

In [ ]:
# 통계적 요약 및 인사이트 도출
print("=" * 60)
print("           대한민국 출생아수 및 합계출산율 분석 보고서")
print("=" * 60)

# 1. 기본 통계 요약
print("\n📊 1. 기본 통계 요약")
print("-" * 40)
if '연도' in df_sorted.columns:
    print(f"분석 기간: {df_sorted['연도'].min()}년 ~ {df_sorted['연도'].max()}년 ({len(df_sorted)}년간)")

if '출생아수' in df_sorted.columns:
    print(f"\n출생아수:")
    print(f"  • 최대: {df_sorted['출생아수'].max():,}명")
    print(f"  • 최소: {df_sorted['출생아수'].min():,}명") 
    print(f"  • 평균: {df_sorted['출생아수'].mean():,.0f}명")
    print(f"  • 표준편차: {df_sorted['출생아수'].std():,.0f}명")
    
    # 출생아수 감소율 계산
    if len(df_sorted) > 1:
        first_value = df_sorted['출생아수'].iloc[0]
        last_value = df_sorted['출생아수'].iloc[-1]
        total_change = ((last_value - first_value) / first_value) * 100
        print(f"  • 전체 기간 변화율: {total_change:+.1f}%")

if '합계출산율' in df_sorted.columns:
    print(f"\n합계출산율:")
    print(f"  • 최대: {df_sorted['합계출산율'].max():.3f}")
    print(f"  • 최소: {df_sorted['합계출산율'].min():.3f}")
    print(f"  • 평균: {df_sorted['합계출산율'].mean():.3f}")
    print(f"  • 표준편차: {df_sorted['합계출산율'].std():.3f}")
    
    # 출산율 감소율 계산
    if len(df_sorted) > 1:
        first_value = df_sorted['합계출산율'].iloc[0]
        last_value = df_sorted['합계출산율'].iloc[-1]
        total_change = ((last_value - first_value) / first_value) * 100
        print(f"  • 전체 기간 변화율: {total_change:+.1f}%")

# 2. 주요 인사이트
print("\n💡 2. 주요 인사이트")
print("-" * 40)

insights = []

# 출산율 수준 분석
if '합계출산율' in df_sorted.columns:
    current_rate = df_sorted['합계출산율'].iloc[-1] if len(df_sorted) > 0 else None
    if current_rate:
        if current_rate < 1.0:
            insights.append(f"현재 합계출산율 {current_rate:.3f}로 극저출산 상태입니다.")
        elif current_rate < 1.3:
            insights.append(f"현재 합계출산율 {current_rate:.3f}로 초저출산 상태입니다.")
        elif current_rate < 2.1:
            insights.append(f"현재 합계출산율 {current_rate:.3f}로 인구대체수준 미달입니다.")
        
        below_replacement = (df_sorted['합계출산율'] < 2.1).sum()
        insights.append(f"전체 기간 중 {below_replacement}년({below_replacement/len(df_sorted)*100:.1f}%)이 인구대체수준 미달입니다.")

# 변화 트렌드 분석
if '출생아수_변화율' in df_sorted.columns:
    negative_years = (df_sorted['출생아수_변화율'] < 0).sum()
    insights.append(f"출생아수가 감소한 연도는 {negative_years}년입니다.")

if '합계출산율_변화율' in df_sorted.columns:
    negative_years = (df_sorted['합계출산율_변화율'] < 0).sum()
    insights.append(f"합계출산율이 감소한 연도는 {negative_years}년입니다.")

# 상관관계 인사이트
if '출생아수' in df_sorted.columns and '합계출산율' in df_sorted.columns:
    corr = df_sorted['출생아수'].corr(df_sorted['합계출산율'])
    if abs(corr) > 0.5:
        direction = "양의" if corr > 0 else "음의"
        insights.append(f"출생아수와 합계출산율 간에는 {direction} 상관관계(r={corr:.3f})가 있습니다.")

for i, insight in enumerate(insights, 1):
    print(f"{i}. {insight}")

# 3. 정책적 시사점
print("\n🏛️ 3. 정책적 시사점")
print("-" * 40)
policy_implications = [
    "저출산 문제 해결을 위한 종합적인 정책 접근이 필요합니다.",
    "육아 지원, 일-가정 양립, 주거 안정 등 다각도의 정책 개발이 요구됩니다.",
    "장기적인 관점에서 지속적인 모니터링과 정책 효과 평가가 중요합니다.",
    "지역별, 연령별 세부 분석을 통한 맞춤형 정책 수립이 필요할 것입니다."
]

for i, implication in enumerate(policy_implications, 1):
    print(f"{i}. {implication}")

# 4. 데이터 품질 평가
print("\n📋 4. 데이터 품질 평가")
print("-" * 40)
print(f"데이터 완성도: {(1 - df_sorted.isnull().sum().sum() / (len(df_sorted) * len(df_sorted.columns))) * 100:.1f}%")
print(f"분석에 사용된 데이터 포인트: {len(df_sorted)}개")
print(f"주요 변수: {', '.join([col for col in ['연도', '출생아수', '합계출산율'] if col in df_sorted.columns])}")

print("\n" + "=" * 60)
print("                        분석 완료")
print("=" * 60)

## 분석 완료

이 노트북에서는 대한민국의 출생아수와 합계출산율 데이터에 대해 다음과 같은 분석을 수행했습니다:

### 수행된 분석
- ✅ 데이터 로드 및 기본 구조 파악
- ✅ 결측값 확인 및 처리
- ✅ 데이터 타입 변환 및 정리
- ✅ 이상값 탐지 및 분석
- ✅ 시계열 트렌드 분석
- ✅ 다양한 시각화 (선그래프, 막대그래프, 히스토그램, 박스플롯 등)
- ✅ 상관관계 분석
- ✅ 통계적 요약 및 인사이트 도출

### 주요 발견사항
- 대한민국의 출생아수와 합계출산율 변화 패턴 분석
- 인구대체수준 대비 현재 상황 평가
- 정책적 시사점 도출

---
**참고**: 이 분석은 제공된 데이터를 기반으로 수행되었습니다. 더 정확한 분석을 위해서는 최신 공식 통계 데이터를 사용하시기 바랍니다.